In [1]:
import dicom
import os
import numpy as np
import glob
import cv2
%matplotlib inline
import matplotlib.pyplot as py
import SimpleITK as sitk
from PIL import Image
from scipy.misc import imrotate

In [2]:
def getAlignImg(t,label = None):#!!!notice, only take uint8 type for the imrotate function!!!
    f = lambda x:np.asarray([float(a) for a in x]);
    o = f(t.ImageOrientationPatient);
    o1 = o[:3];
    o2 = o[3:];
    oh = np.cross(o1,o2);
    or1 = np.asarray([0.6,0.6,-0.2]);
    o2new = np.cross(oh,or1);
    theta = np.arccos(np.dot(o2,o2new)/np.sqrt(np.sum(o2**2)*np.sum(o2new**2)))*180/3.1416;
    print np.arccos(np.dot(o2,o2new)/np.sqrt(np.sum(o2**2)*np.sum(o2new**2)))
    theta = theta * np.sign(np.dot(oh,np.cross(o2,o2new)));
    im_max = np.percentile(t.pixel_array.flatten(),99);
    res = imrotate(np.array(np.clip(np.array(t.pixel_array,dtype=np.float)/im_max*256,0,255),dtype=np.uint8),90);
    if label is None:
        return res;
    else:
        lab = imrotate(label,theta);
        return res,lab

In [3]:
def Orientation_M1 (img):
    if img.InPlanePhaseEncodingDirection=='COL':
        img_new=cv2.transpose(img.pixel_array)
        #py.imshow(img_new)
        img_new = cv2.flip(img_new, 0)
        return img_new
    else:
        #print 'Row Oriented'
        return img.pixel_array

In [4]:
def get_NewOrientation(img_array):
    y, x = np.nonzero(img_array)
    x = x - np.mean(x)
    y = y - np.mean(y)
    coords = np.vstack([x, y])
    cov = np.cov(coords)
    evals, evecs = np.linalg.eig(cov)
    sort_indices = np.argsort(evals)[::-1]
    x_v1, y_v1 = evecs[:, sort_indices[0]]  # Eigenvector with largest eigenvalue
    x_v2, y_v2 = evecs[:, sort_indices[1]]
    return (x_v1,y_v1), (x_v2,y_v2)

In [5]:
L=glob.glob("/Users/egouda/Downloads/train/*/*/*/*")

In [6]:
L=glob.glob("/Users/egouda/Downloads/train/*/*/*/*")
# Kaggle_train = "/Users/laurawilke/Documents/DSE/lwilke/Capstone/Data/Sample data ( kaggle)/*/study/*/*"
#Get all the file names for the Kaggle Images
lstFilesKaggle_train=L
#Get a list of all the paths for the sax images
lstFilesKaggle_train_sax = [s for s in lstFilesKaggle_train if "sax" in s]

In [7]:
len(lstFilesKaggle_train_sax)

161910

In [8]:
img={}
get_align_img=[]
M1=[]

for i in range(37):
    rand=np.random.randint(0,len(lstFilesKaggle_train_sax))
    tmp_img=dicom.read_file(lstFilesKaggle_train_sax[rand])
    img[lstFilesKaggle_train_sax[rand]]=tmp_img
    align=getAlignImg(tmp_img,label=None)
    get_align_img.append(align)
    orient=Orientation_M1(tmp_img)
    M1.append(orient)

0.293075287128
0.455067010622
1.21083894184
1.07000427448
0.269769662896
0.28648796882
0.191959984671
1.14876728404
0.123140074775
0.187995383054
0.00862532632363
0.227258906081
0.255999649497
0.182474050727
0.0727450683364
0.125286643292
0.22114271599
0.817054549631
0.316750764516
0.169530082836
0.11653538867
0.0996775661401
0.258081061252
0.267931650078
0.169153949732
0.130449590981
1.17786740183
1.16783081657
0.314713674303
0.281021538366
0.022822491473
0.0328551479709
0.154611837133
1.27107940405
0.219314132628
0.0258067068491
0.24710345451


In [9]:
M1

[array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 1],
        [0, 0, 0, ..., 0, 1, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ..., 
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint16),
 array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ...,

In [10]:
keys=img.keys()

In [21]:
original=get_NewOrientation(img[keys[0]].pixel_array)
get_align_or=get_NewOrientation(get_align_img[0])
M1_or=get_NewOrientation(M1[0])
print original
print get_align_or
print M1_or

((-0.065161225602084594, -0.99787474899309592), (-0.99787474899309592, 0.065161225602084594))
((0.87384387431585797, -0.4862066261587259), (0.4862066261587259, 0.87384387431585797))
((-0.13381899072357301, -0.99100579096275931), (-0.99100579096275931, 0.13381899072357301))


In [12]:
angle(original[0], get_align_or[0])

NameError: name 'angle' is not defined

In [13]:
original=get_NewOrientation(img[keys[2]].pixel_array)
get_align_or=get_NewOrientation(get_align_img[2])
M1_or=get_NewOrientation(M1[2])
print original
print get_align_or
print M1_or

((0.0075236365458200075, -0.99997169704603461), (-0.99997169704603461, -0.0075236365458200075))
((-0.33210315745326946, -0.94324307196478741), (-0.94324307196478741, 0.33210315745326946))
((0.016969243778263903, -0.99985601201652718), (-0.99985601201652718, -0.016969243778263903))


In [15]:
original=get_NewOrientation(img[keys[8]].pixel_array)
get_align_or=get_NewOrientation(get_align_img[8])
M1_or=get_NewOrientation(M1[8])
print original
print get_align_or
print M1_or

((-0.031323969490264364, -0.99950928406662287), (-0.99950928406662287, 0.031323969490264364))
((0.93574247510053155, -0.35268402330234799), (0.35268402330234799, 0.93574247510053155))
((-0.054118111940873639, -0.99853454119522334), (-0.99853454119522334, 0.054118111940873639))


In [16]:
original=get_NewOrientation(img[keys[5]].pixel_array)
get_align_or=get_NewOrientation(get_align_img[5])
M1_or=get_NewOrientation(M1[5])
print original[0]
print get_align_or
print M1_or

(-0.040494080799142039, -0.99917977832832094)
((0.97244842737810877, -0.2331181161768493), (0.2331181161768493, 0.97244842737810877))
((-0.044160489954171038, -0.99902444971432369), (-0.99902444971432369, 0.044160489954171038))


In [18]:

import math

def dotproduct(v1, v2):
  return sum((a*b) for a, b in zip(v1, v2))

def length(v):
  return math.sqrt(dotproduct(v, v))

def angle(v1, v2):
  return math.acos(dotproduct(v1, v2) / (length(v1) * length(v2)))

In [23]:
angle((-0.065161225602084594, -0.99787474899309592), (-0.13381899072357301, -0.99100579096275931))

0.0690142121543562

In [25]:
math.degrees(0.0690142121543562)

3.954223082865079

In [26]:
192*192

36864

In [27]:
256*256

65536

In [28]:
65536-36864

28672

In [29]:
28672.0/36864

0.7777777777777778